# epaaws example

---
    author: Barron H. Henderson
    date: 2024-04-25
    last-updated: 2024-04-25
---

This shows how to use epaaws to acquire a file, investigate it, and make a plot.

In [ ]:
!python -m pip install -qq epaaws pycno  # install libraries

In [ ]:
# import modeling platform 2022
from epaaws import mp2022
print('\n - '.join(['Currently accessible files:'] + mp2022.__all__))

In [ ]:
# open a GRIDCRO2D file
gf = mp2022.open_gridcro2d('2022-01-01')
gf

In [ ]:
levels = [-100, -10, 1, 100, 200, 400, 800, 1600, 3200]
qm = gf['HT'].plot(levels=levels, cmap='terrain')
gf.csp.cno.drawstates()

# LUFRAC has types in LAY

Numeric indices are stored in VGLVLS and described in 
https://github.com/wrf-model/WRF/blob/master/run/LANDUSE.TBL#L503-L545

In [ ]:
lf = mp2022.open_lufrac_cro('2022-07-15')
lf

In [ ]:
lf.coords['LAY'] = lf.VGLVLS[:-1].astype('<d')
Z = lf['LUFRAC'].sel(LAY=[1, 2, 3, 4, 5, 28, 29, 30]).mean('LAY')
Z.attrs.update(long_name='All Forests', units='1')

Z.plot()
lf.csp.cno.drawstates(resnum=1)

# METCRO2D has lots of variables

* example plot includes surface pressure (PRSFC) and winds

In [ ]:
# open a METCRO2D file
mf = mp2022.open_metcro2d('2022-01-01')
mf

In [ ]:
import numpy as np

wdir = np.radians(mf['WDIR10'])
# in grid length units
mf['U10'] = -mf['WSPD10'] * np.sin(wdir) / mf.XCELL
mf['U10'].attrs.update(long_name='U10', units=f'{mf.XCELL:.1f}')
mf['V10'] = -mf['WSPD10'] * np.cos(wdir) / mf.YCELL
mf['V10'].attrs.update(long_name='V10', units=f'{mf.XCELL:.1f}')

mdf = mf[['PRSFC', 'U10', 'V10']].mean(('TSTEP', 'LAY'), keep_attrs=True)

P = mdf['PRSFC'] / 100
P.attrs['units'] = 'hPa'
qm = P.plot()
mf.csp.cno.drawstates(color='grey')
mdf.plot.streamplot(x='COL', y='ROW', u='U10', v='V10', ax=qm.axes, color='k')
